In [ ]:
import os
import json
import pandas as pd
import traceback
#conda install ipykernel -y this code to resolve below error then i have cloed then i reponed it was worked to cnt ipykernel

: 

In [1]:
import os
import json
import pandas as pd
import traceback

In [11]:
from dotenv import load_dotenv
load_dotenv() # it takes values  .env from our local folder whatever the values key

True

In [12]:
# inside folder structure i m gng to create .env file where i kept my api key
KEY=os.getenv("OPENAI_API_KEY") # this line will pick key in our folder structure

In [15]:
#print(KEY) # here we got our key # if get any error load the env package
# When i run this code it was showin my key so i hidden it and recommit

In [14]:
from langchain.chat_models import ChatOpenAI # in class we use langchain.llm and import openAI this lib also do same things

llm=ChatOpenAI(openai_api_key=KEY,model_name="gbt-3.5-turbo",temperature=0.5) 
# we are going to paste entire key instead do some

In [ ]:
# we gng to implement whtever we learn from past
